## Вариант создания набора координат с 10-мин интервалом за сутки для каждого спутника in-memory в текст

In [21]:
from kafka import KafkaProducer
from json import dumps

In [22]:
producer = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda m: dumps(m).encode('utf-8'))

In [23]:
from datetime import datetime, date, timedelta
#import time
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
from pyorbital.orbital import Orbital
import configparser
import pandas as pd
#import threading


 
config = configparser.ConfigParser()
config.read('config.ini', encoding='utf-8')
user=config.get('spacetrack', 'user')
password=config.get('spacetrack', 'password')

def get_spacetrack_tle (sat_id, start_date, end_date, username, password, latest=False):
    st = SpaceTrackClient(identity=username, password=password)
    if not latest:
        daterange = op.inclusive_range(start_date, end_date)
        data = st.tle(norad_cat_id=sat_id, orderby='epoch desc', limit=1, format='tle', epoch = daterange)
    else:
        data = st.tle_latest(norad_cat_id=sat_id, orderby='epoch desc', limit=1, format='tle')
 
    if not data:
        return 0, 0
 
    tle_1 = data[0:69]
    tle_2 = data[70:139]
    return tle_1, tle_2
 
# На вход будем требовать идентификатор спутника, день (в формате date (y,m,d))
# шаг в минутах для определения положения спутника, путь для результирующего файла
def create_orbital_track_for_day (sat_id, track_day, step_minutes, USERNAME, PASSWORD):
    print('------------Запуск в ' + str(datetime.utcnow()))
    # Для начала получаем TLE    
    # Если запрошенная дата наступит в будущем, то запрашиваем самые последний набор TLE 
    if track_day > date.today():
        tle_1, tle_2 = get_spacetrack_tle (sat_id, None, None, USERNAME, PASSWORD, True)
    # Иначе на конкретный период, формируя запрос для указанной даты и дня после неё
    else:
        tle_1, tle_2 = get_spacetrack_tle (sat_id, track_day, track_day + timedelta(days = 1), USERNAME, PASSWORD, False)
     
    # Если не получилось добыть    
    if not tle_1 or not tle_2:
        print ('Невозможно получить набор данных TLE для sat_id = ' + str(sat_id) + ' for track_day = ' + str(track_day))        
        return
 
    print (tle_1, tle_2)   
    # Создаём экземляр класса Orbital
    orb = Orbital("N", line1=tle_1, line2=tle_2)

    # Объявляем счётчики, i для идентификаторов, minutes для времени
    i = 0
    minutes = 0
    #set_row = 'sat_id,i,utcdatetime,lat,lon' + '\n'
    set_row = ''
    #df = pd.dataframe()
    # Простой способ пройти сутки - с заданным в минутах шагом дойти до 1440 минут.
    while minutes < 1440:
        # Рассчитаем час, минуту, секунду (для текущего шага)
        utc_hour = int(minutes // 60)
        utc_minutes = int((minutes - (utc_hour*60)) // 1)
        utc_seconds = int(round((minutes - (utc_hour*60) - utc_minutes)*60))
 
        # Сформируем строку для атрибута
        utc_string = str(utc_hour) + '-' + str(utc_minutes) + '-' + str(utc_seconds)
        # И переменную с временем текущего шага в формате datetime
        utc_time = datetime(track_day.year,track_day.month,track_day.day,utc_hour,utc_minutes,utc_seconds)
        # Считаем положение спутника
        lon, lat, alt = orb.get_lonlatalt(utc_time)
 
        # запишем
        if i == 0:
            set_row = str(sat_id) + ',' + str(i) + ',' + '\'' +  str(utc_time)  + '\'' + ',' + str(lat) + ',' + str(lon)
        else:
            set_row = set_row + '\n' + str(sat_id) + ',' + str(i) + ',' + '\'' + str(utc_time) + '\'' + ',' + str(lat) + ',' + str(lon)
        #writer.writerow(sat_id, track_day, i, utc_string, utc_time, lat, lon])
        # Не забываем про счётчики
        i += 1
        minutes += step_minutes
 
    try:
        if len(set_row) > 5:
            print(set_row)
            #отправить в кафку
            
            producer.send('satellite_msg', value=set_row)
            print('-------------------------send to kafka')
            print('-------------------------sleep 1')
            time.sleep(1)
        print('------------Финиш для ' + str(sat_id) + ' в ' + str(track_day) +'  ' + str(datetime.utcnow()) + '\n')
    except Exception as e:
        print ('Невозможно отправить данные в кафку, ошибка: ' + str(e))
    return

In [26]:
utc_time = datetime.utcnow()
#utc_time = date(2020,6,26) #загрузим разово вчерашний день
#sat_list_id = [41444, 20887, 20196, 20671, 41258, 20670, 21750, 41257, 20150, 41255, 45124, 45123, 45122, 20666, 45109, 20625, 20032, 20029, 41252, 45090, 41240, 20608, 20580, 45037, 41224, 41221, 45021, 45018, 45014, 45011, 45010, 45007, 44999, 20565, 44996, 44995, 44988]
#sat_list_id = [32221,32218,32212,32211,32203,5150,32197,4842,32194,32193,5149,32184,32181,32179,5129,32168,32165,32163,4822,35243,36076,32148,32143,4777,4766,32130]
#sat_list_id = [25544, 2883, 22377]

# Получение списка низкоорбитальных ИСЗ (LEO)  15369 строк
sat_list = pd.read_csv('./data_in/satellite_list.csv', sep=",")
sat_list_id = sat_list['NORAD_CAT_ID']


for sat_id in sat_list_id:
    try:
        #path_csv = '../data_out/' + str(sat_id) + '_' + str(date(utc_time.year,utc_time.month,utc_time.day)) +  '.csv'
        print('Поиск данных для спутника ' + str(sat_id) + ' на дату ' + str(date(utc_time.year,utc_time.month,utc_time.day)))
        create_orbital_track_for_day(sat_id, date(utc_time.year,utc_time.month,utc_time.day), 10, user, password)
    except Exception as e:
        print ('Невозможно извлечь данные, ошибка: ' + str(e))
    continue

Поиск данных для спутника 32221 на дату 2020-06-28
------------Запуск в 2020-06-28 14:12:42.519244
Невозможно получить набор данных TLE для sat_id = 32221 for track_day = 2020-06-28
Поиск данных для спутника 32218 на дату 2020-06-28
------------Запуск в 2020-06-28 14:12:45.651888
1 32218U 99025CVQ 20180.14541847  .00000132  00000-0  57036-4 0  9997 2 32218  98.6584 292.7357 0082611 269.1577 212.4388 14.38111783676271
32218,0,'2020-06-28 00:00:00',81.34774924537444,-68.49106922320114
32218,1,'2020-06-28 00:10:00',54.60903340885593,-154.1525736877301
32218,2,'2020-06-28 00:20:00',19.628376799932834,-165.86660013890375
32218,3,'2020-06-28 00:30:00',-16.077494732369985,-173.96505094221388
32218,4,'2020-06-28 00:40:00',-51.732365750734324,174.96708210860834
32218,5,'2020-06-28 00:50:00',-81.3084409931282,101.32134135861155
32218,6,'2020-06-28 01:00:00',-54.01103384832115,13.07505482717412
32218,7,'2020-06-28 01:10:00',-18.408010803157538,1.4269851073727011
32218,8,'2020-06-28 01:20:00',17.3

------------Финиш для 32218 в 2020-06-28  2020-06-28 14:12:49.770631

Поиск данных для спутника 32212 на дату 2020-06-28
------------Запуск в 2020-06-28 14:12:49.771647
Невозможно получить набор данных TLE для sat_id = 32212 for track_day = 2020-06-28
Поиск данных для спутника 32211 на дату 2020-06-28
------------Запуск в 2020-06-28 14:12:51.889539
Невозможно получить набор данных TLE для sat_id = 32211 for track_day = 2020-06-28
Поиск данных для спутника 32203 на дату 2020-06-28
------------Запуск в 2020-06-28 14:12:53.889021
Невозможно получить набор данных TLE для sat_id = 32203 for track_day = 2020-06-28
Поиск данных для спутника 5150 на дату 2020-06-28
------------Запуск в 2020-06-28 14:12:55.979898
Невозможно получить набор данных TLE для sat_id = 5150 for track_day = 2020-06-28
Поиск данных для спутника 32197 на дату 2020-06-28
------------Запуск в 2020-06-28 14:12:58.231780
Невозможно получить набор данных TLE для sat_id = 32197 for track_day = 2020-06-28
Поиск данных для спутн

------------Финиш для 5149 в 2020-06-28  2020-06-28 14:13:10.964349

Поиск данных для спутника 32184 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:10.964631
Невозможно получить набор данных TLE для sat_id = 32184 for track_day = 2020-06-28
Поиск данных для спутника 32181 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:12.968196
1 32181U 99025CUB 20180.03649163  .00000250  00000-0  89839-4 0  9991 2 32181  99.5225  20.1829 0094236 173.5751 340.5108 14.41392243679785
32181,0,'2020-06-28 00:00:00',-34.36237746613956,110.1407563230008
32181,1,'2020-06-28 00:10:00',0.5950316057373574,100.99116612186975
32181,2,'2020-06-28 00:20:00',35.63124083140693,91.7225473666968
32181,3,'2020-06-28 00:30:00',69.63364312369492,69.38295627087466
32181,4,'2020-06-28 00:40:00',69.86598456674992,-59.33577925227295
32181,5,'2020-06-28 00:50:00',35.12329185335538,-82.16621059544569
32181,6,'2020-06-28 01:00:00',-1.1744340288802684,-91.60107898448598
32181,7,'2020-06-28 01:10:00',-37.3433646

------------Финиш для 32181 в 2020-06-28  2020-06-28 14:13:17.109182

Поиск данных для спутника 32179 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:17.109481
Невозможно получить набор данных TLE для sat_id = 32179 for track_day = 2020-06-28
Поиск данных для спутника 5129 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:19.108455
Невозможно получить набор данных TLE для sat_id = 5129 for track_day = 2020-06-28
Поиск данных для спутника 32168 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:21.164370
Невозможно получить набор данных TLE для sat_id = 32168 for track_day = 2020-06-28
Поиск данных для спутника 32165 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:23.413134
Невозможно получить набор данных TLE для sat_id = 32165 for track_day = 2020-06-28
Поиск данных для спутника 32163 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:25.556756
Невозможно получить набор данных TLE для sat_id = 32163 for track_day = 2020-06-28
Поиск данных для спутн

------------Финиш для 35243 в 2020-06-28  2020-06-28 14:13:35.000881

Поиск данных для спутника 36076 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:35.001174
Невозможно получить набор данных TLE для sat_id = 36076 for track_day = 2020-06-28
Поиск данных для спутника 32148 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:37.030826
Невозможно получить набор данных TLE для sat_id = 32148 for track_day = 2020-06-28
Поиск данных для спутника 32143 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:42.238745
Невозможно получить набор данных TLE для sat_id = 32143 for track_day = 2020-06-28
Поиск данных для спутника 4777 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:44.392573
Невозможно получить набор данных TLE для sat_id = 4777 for track_day = 2020-06-28
Поиск данных для спутника 4766 на дату 2020-06-28
------------Запуск в 2020-06-28 14:13:46.527222
Невозможно получить набор данных TLE для sat_id = 4766 for track_day = 2020-06-28
Поиск данных для спутник

In [13]:
f = '2020-06-28 00:00:00'
datetime.strptime(f, '%Y-%m-%d %H:%M:%S')

datetime.datetime(2020, 6, 28, 0, 0)